### 네이버 영화평 감성 분석

In [3]:
!pip install konlpy > /dev/null
# dev가 휴지통 같은 것. 화면에서 나오는것 다 휴지통으로 보냄

In [4]:
# !pip install konlpy > /dev/null 2>&1
# 2>&1 쓰면 에러도 안나옴

In [5]:
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [25]:
# 네이버 영화 리뷰 데이터로 검색
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')

In [26]:
print(train_df.shape, test_df.shape)
train_df.head(3)

(150000, 3) (50000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


1. 데이터 전처리
 - 트레인 데이터셋

In [27]:
# Null 데이터가 있는지 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [28]:
# Null 데이터 제거
train_df.dropna(how='any', inplace=True)
train_df.shape

(149995, 3)

In [29]:
# 중복 여부 확인
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146182, 3)

In [30]:
# 긍정(1)/부정(0) 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터셋

In [31]:
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [32]:
test_df.dropna(how='any', inplace=True)


In [33]:
test_df.document.nunique()

49157

In [34]:
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [35]:
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

### 2. 텍스트 전처리
 - 트레인 데이터셋

In [37]:
# 한글 이외의 문자는 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', ).str.strip()
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [38]:
# ''만 남은 데이터 --> np.nan 으로 대체한 후 제거
train_df.document.replace('', np.nan, inplace=True)
train_df.document.isna().sum()

789

In [39]:
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 3)

In [40]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', ).str.strip()
test_df.document.replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48852, 3)

- 전처리가 끝난 데이터는 저장

In [41]:
train_df.to_csv('naver_movie_train_전처리완료.tsv', sep='\t', index=False)
test_df.to_csv('naver_movie_test_전처리완료.tsv', sep='\t', index=False)

In [42]:
from konlpy.tag import Okt
okt = Okt()

In [44]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [46]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']
' '.join([word for word in okt.morphs(text, stem=True) if word not in stopwords])


'교도소 이야기 구먼 솔직하다 재미 없다 평점 조정'

In [ ]:
from tqdm.notebook import tqdm

X_train = []
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_str = ' '.join([word for word in morphs if word not in stopwords])
    X_train.append(tmp_str)

X_train

In [ ]:
X_test = []
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_str = ' '.join([word for word in morphs if word not in stopwords])
    X_test.append(tmp_str)

X_test

In [ ]:
# pipeline으로 countvectorizer + logisticregression 학습

In [52]:
y_train = train_df.label.values
y_test = test_df.label.values

### 4. 피쳐 변환 + 모델 학습

In [51]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression


In [53]:
cvect = CountVectorizer()
lr = LogisticRegression(random_state=2022)
pipeline = Pipeline([
                     ('CVECT', cvect), ('LR', lr)
])
%time pipeline.fit(X_train, y_train)

Pipeline(steps=[('CVECT', CountVectorizer()),
                ('LR', LogisticRegression(random_state=2022))])

In [54]:
pipeline.score(X_test, y_test)

0.8273356259723246

In [58]:
review1 = '모든 국민이 봤으면 하는 영화입니다'
review2 = '생각보다 지루하고 별로였네요... 보면서 좀 졸았습니다... ㅜ'

In [59]:
# 전처리
import re
review1 = re.sub('[^가-힣]', ' ', review1)
review2 = re.sub('[^가-힣]', ' ', review2)

In [60]:
morphs = okt.morphs(review1, stem=True)
review1 = ' '.join([word for word in morphs if word not in stopwords])
morphs = okt.morphs(review2, stem=True)
review2 = ' '.join([word for word in morphs if word not in stopwords])

In [61]:
pipeline.predict([review1, review2])

array([1, 0])

### 6. 최적 파라메터 찾기

In [63]:
from sklearn.model_selection import GridSearchCV
params = {
    'CVECT__ngram_range' : [(1,1), (1,2)],
    'CVECT__max_df' : [0.95, 0.98],
    'LR__C' : [1, 5]
}
grid_pipe = GridSearchCV(
    pipeline, params, scoring='accuracy', cv=3, n_jobs=-1
)
%time grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)
best_pipe = grid_pipe.best_estimator_
best_pipe.score(X_test, y_test)

CPU times: user 1min 7s, sys: 22.2 s, total: 1min 29s
Wall time: 4min 38s
{'CVECT__max_df': 0.95, 'CVECT__ngram_range': (1, 2), 'LR__C': 1} 0.8418493092569369


0.8479898468844673